In [1]:
import torch as th
import torch
import torch.nn as nn
import time
import math
import copy

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
class TCNN(th.nn.Module):
    def __init__(self, kernel_size, channels, ntoken) -> None:
        super().__init__()
        self.model = th.nn.ModuleList()
        self.encoder = nn.Linear(1, channels[0])
        #self.encoder = nn.Embedding(ntoken, channels[0])
        self.decoder = nn.Linear(channels[-1], ntoken)
        for i in range(len(channels)-1):
            self.model.append(th.nn.Conv1d(channels[i], channels[i+1], kernel_size=kernel_size, stride=1))
            if i < len(channels) - 2:
                #self.model.append(th.nn.BatchNorm1d(num_features=channels[i+1]))
                self.model.append(th.nn.ReLU())
    
    def forward(self, input:th.Tensor):
        embedded = self.encoder.forward(input)
        internal_result = embedded.permute(1,2,0)
        for module in self.model:
            internal_result = module.forward(internal_result)
                
        internal_result = internal_result.permute(2,0,1)
        result = self.decoder(internal_result)
        return result

In [56]:
seq_len = 100
batch_size = 2
ntoken=100
kernel_size = 35

tcnn = TCNN(seq_len=kernel_size, channels=[10,10, 10], ntoken=ntoken)

inpt = th.ones([seq_len, batch_size, 1], dtype=th.float, requires_grad=True)
result = tcnn.forward(inpt)

In [57]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter):
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data, bsz: int):
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size).to('cuda')  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size).to('cuda')
test_data = batchify(test_data, eval_batch_size).to('cuda')

bptt = kernel_size + kernel_size - 1
def get_batch(source, i: int):
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source)- (2*bptt) - i)
    data = source[i:i+seq_len]
    target = source[i+seq_len:i+(2*seq_len)]
    return data, target

In [14]:
batch_size = 2
seq_len = bptt
dim = 3
ntokens = len(vocab)  # size of vocabulary
inpt = th.ones([batch_size, dim, seq_len])
model = TCNN(seq_len=seq_len, channels=[180, 180, 180], ntoken=ntokens)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model=model)

12659022

In [16]:
model = model.to('cuda')

In [20]:
model

TCNN(
  (model): ModuleList(
    (0): Conv1d(180, 180, kernel_size=(35,), stride=(1,))
    (1): BatchNorm1d(180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(180, 180, kernel_size=(35,), stride=(1,))
  )
  (encoder): Embedding(28782, 180)
  (decoder): Linear(in_features=180, out_features=28782, bias=True)
)

In [17]:
train_data.device

device(type='cuda', index=0)

In [18]:
criterion = nn.CrossEntropyLoss()
lr = 5  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    
    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            break
        print(data.shape)
        output = model.forward(data)
        loss = criterion(output.view(-1, ntokens), targets.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                break
            output = model(data)
            output_flat = output[-1].view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets[-1].reshape(-1)).item()
    return total_loss / (len(eval_data) - 1)

In [19]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

torch.Size([35, 20])
asd: torch.Size([20, 180, 35])
<class 'torch.nn.modules.conv.Conv1d'>
asd: torch.Size([20, 180, 1])
<class 'torch.nn.modules.batchnorm.BatchNorm1d'>
asd: torch.Size([20, 180, 1])
<class 'torch.nn.modules.activation.ReLU'>
asd: torch.Size([20, 180, 1])
<class 'torch.nn.modules.conv.Conv1d'>


RuntimeError: Calculated padded input size per channel: (1). Kernel size: (35). Kernel size can't be greater than actual input size

In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  6.06 | test ppl   430.01
